In [4]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [5]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")


OpenAI API Key not set
Anthropic API Key exists and begins sk-ant-
Google API Key not set (and this is optional)
DeepSeek API Key not set (and this is optional)
Groq API Key not set (and this is optional)
Grok API Key not set (and this is optional)
OpenRouter API Key exists and begins sk-


In [6]:
# Using open router with any model
from openai import OpenAI

client = OpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

easy_puzzle = [
    {"role": "user", "content": 
        "You toss 2 coins. One of them is heads. What's the probability the other is tails? Answer with the probability only."},
]

response = client.chat.completions.create(
    model    = "anthropic/claude-sonnet-4.6",   # ← change only here
    messages = easy_puzzle,
)

print(response.choices[0].message.content)

2/3


In [ ]:
#Testing various models with same question

from openai import OpenAI


openai_client = OpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

hard = """
On a bookshelf, two volumes of Pushkin stand side by side: the first and the second.
The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick.
A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume.
What distance did it gnaw through?
"""
hard_puzzle = [
    {"role": "user", "content": hard}
]



## Using openai/gpt-4o-mini

response = openai_client.chat.completions.create(
    model="openai/gpt-4o-mini",
    messages=hard_puzzle, reasoning_effort="minimal"
)

display(Markdown(response.choices[0].message.content))

print("Using google/gemini-2.0-flash-lite.........")


## Using google/gemini-2.0-flash-lite-001   

response = openai_client.chat.completions.create(
    model="google/gemini-2.0-flash-lite-001",
    messages=hard_puzzle, reasoning_effort="minimal"
)

display(Markdown(response.choices[0].message.content))


print("Using anthropic/claude-3-5-sonnet-20240620.........")


## Using anthropic/claude-haiku-4-5
response = openai_client.chat.completions.create(
    model="anthropic/claude-haiku-4-5",
    messages=hard_puzzle, reasoning_effort="minimal"
)

display(Markdown(response.choices[0].message.content)) 

In [8]:
requests.get("http://localhost:11434/").content

b'Ollama is running'

In [ ]:

easy_puzzle = [
    {"role": "user", "content": 
        "You toss 2 coins. One of them is heads. What's the probability the other is tails? Answer with the probability only."},
]
response = ollama.chat.completions.create(model="llama3.2", messages=easy_puzzle)
display(Markdown(response.choices[0].message.content))

In [11]:

from anthropic import Anthropic

client = Anthropic()

response = client.messages.create(
    model="claude-sonnet-4-5-20250929",
    messages=[{"role": "user", "content": "Describe the color Blue to someone who's never been able to see in 1 sentence"}],
    max_tokens=100
)
print(response.content[0].text)

Blue is the color that feels like cool water on your skin, sounds like calm ocean waves, and carries the peaceful quiet of early morning air.


In [18]:
import os

# Point LangChain(OpenAI) to OpenRouter
if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY is not set")


from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="openai/gpt-4o-mini")

tell_a_joke="Tell a joke for a student on the journey to becoming an expert in LLM Engineering"
response = llm.invoke(tell_a_joke)

display(Markdown(response.content))

Why did the LLM engineer break up with their dataset?

Because it had too many "issues" and couldn't stop "overfitting" on past relationships!

## Lightweight LiteLLM

In [25]:
from litellm import completion

if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY is not set")

# LiteLLM expects OpenAI-style chat messages (list of dicts)
messages = tell_a_joke
if isinstance(messages, str):
    messages = [{"role": "user", "content": messages}]

response = completion(
    model="openai/gpt-4o-mini",
    messages=messages,
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

reply = response.choices[0].message.content
display(Markdown(reply))

Why did the language model go to therapy?

Because it had too many unresolved prompts!

In [ ]:
# ---- Token and Cost Reporting ----
# Extract usage information (token counts) from the response object.
# 'usage' can be either a dict (OpenAI-style) or an object with attributes (LiteLLM/other wrappers).
usage = getattr(response, "usage", None) or {}


# Safely retrieve prompt, completion, and total tokens regardless of usage type.
prompt_tokens = usage.get("prompt_tokens") if isinstance(usage, dict) else getattr(usage, "prompt_tokens", None)
completion_tokens = usage.get("completion_tokens") if isinstance(usage, dict) else getattr(usage, "completion_tokens", None)
total_tokens = usage.get("total_tokens") if isinstance(usage, dict) else getattr(usage, "total_tokens", None)

# Print token usage statistics for the request
print(f"Input tokens: {prompt_tokens}")        # Number of tokens in the prompt (input)
print(f"Output tokens: {completion_tokens}")   # Number of tokens generated by the model (output)
print(f"Total tokens: {total_tokens}")         # Combined input and output tokens

# Extract cost information if present.
# Some LiteLLM responses include a hidden dict "_hidden_params" holding "response_cost" (float, in dollars).
hidden = getattr(response, "_hidden_params", None) or {}
response_cost = hidden.get("response_cost") if isinstance(hidden, dict) else None

# Print total cost in cents if available, otherwise print 'n/a'.
if response_cost is None:
    print("Total cost: n/a")
else:
    print(f"Total cost: {response_cost * 100:.4f} cents")

Input tokens: 24
Output tokens: 17
Total tokens: 41
Total cost: 0.0014 cents


## Prompt caching

In [ ]:
with open("hamlet.txt", "r", encoding="utf-8") as f:
    hamlet = f.read()

#Location index where except is found
loc = hamlet.find("Speak, man")
print(loc)
print(hamlet[loc:loc+100])


8
Speak, man
by William Shakespeare

[Public domain]

ACT I
SCENE I. Elsinore. A platform before the c


In [38]:
question = [{"role": "user", "content": "In Hamlet, when Laertes asks 'Where is my father?' what is the reply?"}]

In [40]:
response = completion(
    model="openai/gpt-4o-mini",
    messages=question,
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

display(Markdown(response.choices[0].message.content))

In Shakespeare's "Hamlet," when Laertes asks "Where is my father?", he is referring to Polonius, his father. The response comes from Gertrude, who is shocked and distressed. In Act IV, Scene 5, she tells Laertes that Polonius is dead. This moment is significant as it sets off a chain of events that lead to further conflict in the play. If you would like more details about the context or themes surrounding this moment, feel free to ask!

In [42]:
# ---- Token and Cost Reporting ----
# Extract usage information (token counts) from the response object.
# 'usage' can be either a dict (OpenAI-style) or an object with attributes (LiteLLM/other wrappers).
usage = getattr(response, "usage", None) or {}


# Safely retrieve prompt, completion, and total tokens regardless of usage type.
prompt_tokens = usage.get("prompt_tokens") if isinstance(usage, dict) else getattr(usage, "prompt_tokens", None)
completion_tokens = usage.get("completion_tokens") if isinstance(usage, dict) else getattr(usage, "completion_tokens", None)
total_tokens = usage.get("total_tokens") if isinstance(usage, dict) else getattr(usage, "total_tokens", None)

# Print token usage statistics for the request
print(f"Input tokens: {prompt_tokens}")        # Number of tokens in the prompt (input)
print(f"Output tokens: {completion_tokens}")   # Number of tokens generated by the model (output)
print(f"Total tokens: {total_tokens}")         # Combined input and output tokens

# Extract cost information if present.
# Some LiteLLM responses include a hidden dict "_hidden_params" holding "response_cost" (float, in dollars).
hidden = getattr(response, "_hidden_params", None) or {}
response_cost = hidden.get("response_cost") if isinstance(hidden, dict) else None

# Print total cost in cents if available, otherwise print 'n/a'.
if response_cost is None:
    print("Total cost: n/a")
else:
    print(f"Total cost: {response_cost * 100:.4f} cents")

Input tokens: 25
Output tokens: 102
Total tokens: 127
Total cost: 0.0065 cents


In [54]:
question[0]["content"] += "\n\nFor context, here is the entire text of Hamlet:\n\n"+hamlet

print(question)

[{'role': 'user', 'content': "In Hamlet, when Laertes asks 'Where is my father?' what is the reply?\n\nFor context, here is the entire text of Hamlet:\n\nHAMLET  Speak, man\nby William Shakespeare\n\n[Public domain]\n\nACT I\nSCENE I. Elsinore. A platform before the castle.\n\nFRANCISCO at his post. Enter to him BERNARDO.\n\nBERNARDO:\nWho's there?\n\nFRANCISCO:\nNay, answer me. Stand, and unfold yourself.\n\nBERNARDO:\nLong live the King!\n\nFRANCISCO:\nBernardo?\n\nBERNARDO:\nHe.\n\nFRANCISCO:\nYou come most carefully upon your hour.\n\nBERNARDO:\n'Tis now struck twelve. Get thee to bed, Francisco.\n\nFor context, here is the entire text of Hamlet:\n\nHAMLET  Speak, man\nby William Shakespeare\n\n[Public domain]\n\nACT I\nSCENE I. Elsinore. A platform before the castle.\n\nFRANCISCO at his post. Enter to him BERNARDO.\n\nBERNARDO:\nWho's there?\n\nFRANCISCO:\nNay, answer me. Stand, and unfold yourself.\n\nBERNARDO:\nLong live the King!\n\nFRANCISCO:\nBernardo?\n\nBERNARDO:\nHe.\n\nFR

In [52]:
response = completion(model="openai/gpt-4o-mini",
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1", messages=question)
display(Markdown(response.choices[0].message.content))

In "Hamlet," when Laertes asks, "Where is my father?" he is referring to Polonius. He receives the reply, "At home, my lord," from Ophelia. This exchange occurs in Act 2, Scene 1, when Laertes has just returned to Denmark and is inquiring about his father.

In [53]:
# ---- Token and Cost Reporting ----
# Extract usage information (token counts) from the response object.
# 'usage' can be either a dict (OpenAI-style) or an object with attributes (LiteLLM/other wrappers).
usage = getattr(response, "usage", None) or {}


# Safely retrieve prompt, completion, and total tokens regardless of usage type.
prompt_tokens = usage.get("prompt_tokens") if isinstance(usage, dict) else getattr(usage, "prompt_tokens", None)
completion_tokens = usage.get("completion_tokens") if isinstance(usage, dict) else getattr(usage, "completion_tokens", None)
total_tokens = usage.get("total_tokens") if isinstance(usage, dict) else getattr(usage, "total_tokens", None)

# Print token usage statistics for the request
print(f"Input tokens: {prompt_tokens}")        # Number of tokens in the prompt (input)
print(f"Output tokens: {completion_tokens}")   # Number of tokens generated by the model (output)
print(f"Total tokens: {total_tokens}")         # Combined input and output tokens

# Extract cost information if present.
# Some LiteLLM responses include a hidden dict "_hidden_params" holding "response_cost" (float, in dollars).
hidden = getattr(response, "_hidden_params", None) or {}
response_cost = hidden.get("response_cost") if isinstance(hidden, dict) else None

# Print total cost in cents if available, otherwise print 'n/a'.
if response_cost is None:
    print("Total cost: n/a")
else:
    print(f"Total cost: {response_cost * 100:.4f} cents")

Input tokens: 161
Output tokens: 69
Total tokens: 230
Total cost: 0.0066 cents


In [55]:
# Let's make a conversation between GPT-4.1-mini and Claude-haiku-4.5
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4.1-mini"
claude_model = "claude-haiku-4-5"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [ ]:
from openai import OpenAI

def call_gpt():
    # Initialize the OpenAI client (uses OPENAI_API_KEY from your environment)
    client = OpenAI()

    # Prepare the conversation as a list of messages
    messages = [{"role": "system", "content": gpt_system}]
    # Loop over pairs of messages from the GPT and Claude chatbots.
    # For each turn, 'gpt' is the GPT-4.1-mini response and 'claude' is the Claude-haiku-4.5 response.
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        print(messages)

    # Create the chat completion using the specified model and messages
    response = client.chat.completions.create(model=gpt_model, messages=messages)
    return response.choices[0].message.content

In [65]:
call_gpt()

[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]


In [72]:

import os
from openai import OpenAI

def call_claude():
    # Use the OpenRouter API key from the environment or .env file
    openrouter_api_key = os.environ.get("OPENROUTER_API_KEY")
    client = OpenAI(
        api_key=openrouter_api_key,
        base_url="https://openrouter.ai/api/v1"
    )

    messages = [{"role": "system", "content": claude_system}]
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})

    # Use OpenRouter's model identifier for Claude Haiku 4.5
    response = client.chat.completions.create(
        model="anthropic/claude-3-haiku:beta",
        messages=messages
    )
    return response.choices[0].message.content


In [77]:
call_claude()

'Hello! How are you doing today?'

In [78]:
call_gpt()

[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]


In [79]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

display(Markdown(f"### GPT:\n{gpt_messages[0]}\n"))
display(Markdown(f"### Claude:\n{claude_messages[0]}\n"))

for i in range(5):
    gpt_next = call_gpt()
    display(Markdown(f"### GPT:\n{gpt_next}\n"))
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    display(Markdown(f"### Claude:\n{claude_next}\n"))
    claude_messages.append(claude_next)

### GPT:
Hi there


### Claude:
Hi


[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]


### GPT:
None


### Claude:
 there! How are you doing today? I'm here to chat and help out however I can.


...

### GPT:
None


### Claude:



[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}]
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}, {'role': 'assistant', 'content': None}, {'role': 'user', 'content': " there! How are you doing today? I'm here to chat and help out however I can."}]
[{'role': 'system', 'content': 'You are a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.'}, {'role': 'assistant', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi'}, {'role': 'assistant', 'content': None}, {'role': 'user', 'content': " there! How are 

### GPT:
None


### Claude:

